In [ ]:
! pip install --quiet environs facebook_business cyksuid toolz psycopg2-binary typing_json backoff xxhash

In [ ]:
! pip install --quiet git+https://github.com/SlavaSkvortsov/facebook-python-business-sdk@fix_bug_with_pagination

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from environs import Env
env = Env()
env.read_env('.env')

In [5]:
from malaria import get_conf, get_df, lookalike
from marketing import Marketing

cnf = get_conf(env)
m = Marketing(env)

TypeError: list indices must be integers or slices, not str

In [2]:
from malaria import update_audience, update_ads

# update_audience()

update_ads()

INFO:root:Intiailized Marketing with 0 creatives, 1 running ads and campaign impact-evaluation-vlab-9


/home/jupyter/.local/lib/python3.8/site-packages/facebook_business/utils/api_utils.py:30: UserWarning: value of adlabels might not be compatible.  Expect list<Object>;  got <class 'list'>
  warnings.warn(message)


INFO:root:Facebook error code: 100


Backing off create_creative(...) for 49.1s (facebook_business.exceptions.FacebookRequestError: 

  Message: Call was not successful
  Method:  POST
  Path:    https://graph.facebook.com/v7.0/act_2240415876083895/adcreatives
  Params:  {'name': 'vlab-mnm-hindi-mobile-credit', 'url_tags': 'ref=form.langchoice.clusterid.06622dd4.clustername.Kaushambi', 'actor_id': '100450211748271', 'object_story_spec': '{"instagram_actor_id":"3922044794536181","link_data":{"call_to_action":{"type":"MESSAGE_PAGE","value":{"app_destination":"MESSENGER"}},"image_hash":"dd8fd1497663d46fb64cb37d6f69f5fd","message":"Aapki rai ke lie 300 INR ke mobile credit!","name":"Messenger main survey shuroo karen!","page_welcome_message":"{\\"message\\": {\\"attachment\\": {\\"payload\\": {\\"buttons\\": [{\\"payload\\": \\"{\\\\\\"referral\\\\\\": {\\\\\\"ref\\\\\\": \\\\\\"form.langchoice.clusterid.06622dd4.clustername.Kaushambi\\\\\\"}}\\", \\"title\\": \\"Start!\\", \\"type\\": \\"postback\\"}], \\"template_type\\": \

INFO:backoff:Backing off create_creative(...) for 49.1s (facebook_business.exceptions.FacebookRequestError: 

  Message: Call was not successful
  Method:  POST
  Path:    https://graph.facebook.com/v7.0/act_2240415876083895/adcreatives
  Params:  {'name': 'vlab-mnm-hindi-mobile-credit', 'url_tags': 'ref=form.langchoice.clusterid.06622dd4.clustername.Kaushambi', 'actor_id': '100450211748271', 'object_story_spec': '{"instagram_actor_id":"3922044794536181","link_data":{"call_to_action":{"type":"MESSAGE_PAGE","value":{"app_destination":"MESSENGER"}},"image_hash":"dd8fd1497663d46fb64cb37d6f69f5fd","message":"Aapki rai ke lie 300 INR ke mobile credit!","name":"Messenger main survey shuroo karen!","page_welcome_message":"{\\"message\\": {\\"attachment\\": {\\"payload\\": {\\"buttons\\": [{\\"payload\\": \\"{\\\\\\"referral\\\\\\": {\\\\\\"ref\\\\\\": \\\\\\"form.langchoice.clusterid.06622dd4.clustername.Kaushambi\\\\\\"}}\\", \\"title\\": \\"Start!\\", \\"type\\": \\"postback\\"}], \\"templa

KeyboardInterrupt: 

In [15]:
from datetime import datetime, timezone
from clustering import only_target_users

def filter_time(df, min_date, max_date):
    users = df[(df.timestamp > min_date) & (df.timestamp < max_date)].userid.unique()
    return df[df.userid.isin(users)]

def get_percentage(df, cnf):
    answered = df[df.question_ref == 'f6e69027-97cc-494e-8d52-318b75047e23'].userid.unique()
    df = df[df.userid.isin(answered)]
    tot = df.userid.unique().shape[0]
    targets = only_target_users(df, cnf['strata'][0]['surveys'], 'target_questions')
    targets = targets.userid.unique().shape[0]
    return targets/tot, targets, tot


In [ ]:
day = filter_time(df, datetime(2020,7,23,tzinfo=timezone.utc), datetime(2020,7,25,tzinfo=timezone.utc))

get_percentage(day, cnf)

In [22]:

day = filter_time(df, datetime(2020,7,25,tzinfo=timezone.utc), datetime(2020,7,26,tzinfo=timezone.utc))
get_percentage(day, cnf)

(0.1910828025477707, 270, 1413)

In [23]:
day = filter_time(df, datetime(2020,7,26,tzinfo=timezone.utc), datetime(2020,7,27,tzinfo=timezone.utc))
get_percentage(day, cnf)

(0.16407355021216408, 232, 1414)

In [62]:
day = filter_time(df, datetime(2020,7,27,tzinfo=timezone.utc), datetime(2020,7,28,tzinfo=timezone.utc))
 
get_percentage(day, cnf)

(0.17543859649122806, 20, 114)

In [63]:
get_percentage(df, cnf)

(0.17040125065138093, 981, 5757)

In [59]:
(df[df.question_ref == 'md:clusterid'].response.value_counts() >= 200).sum(), (df[df.question_ref == 'md:clusterid'].response.value_counts() >= 12).sum()

(3, 110)

In [30]:
targets = only_target_users(df, cnf['strata'][0]['surveys'])
users = targets.userid.unique()

In [31]:
(targets[targets.question_ref == 'md:clusterid'].response.value_counts() >= 30).sum(), (targets[targets.question_ref == 'md:clusterid'].response.value_counts() >= 8).sum()

(4, 49)

In [26]:
import pandas as pd
from clustering import add_res_cols
from malaria import load_cities


def perc_target(users, df):
    u = pd.Series(df.userid.unique())
    tot = u.unique().shape[0]
    target = u.isin(users).sum()
    perc = target / tot
    return pd.Series([perc, tot, target], index=['perc', 'tot', 'target'])


def make_analysis_df(df, districts, users):
    finished = df[df.question_ref == 'f6e69027-97cc-494e-8d52-318b75047e23'].userid.unique()
    df = df[df.userid.isin(finished)].reset_index(drop=True)
    df = add_res_cols('md:clusterid', 'cluster', 'userid', df)
    perc = df.groupby('cluster').apply(lambda df: perc_target(users, df)).reset_index()
    perc = perc.merge(districts, left_on='cluster', right_on='disthash')
    return perc

In [7]:

cities = load_cities('output/cities.csv')

districts = cities.groupby('distname') \
                  .head(1) \
                  .reset_index(drop=True) \
                  .drop(columns=['rad', 'lng', 'lat', 'distcode', 'id'])


# perc = make_analysis_df(df, districts, users)

In [10]:
cities[cities.disthash == '06622dd4']

,id,distcode,distname,state,rad,lng,lat,disthash,creative_group
320,245734674,174.0,Kaushambi,Uttar Pradesh,9.875854,81.374938,25.530620,06622dd4,hindi
321,2087323520,174.0,Kaushambi,Uttar Pradesh,5.108331,81.489957,25.561562,06622dd4,hindi
322,2146977936,174.0,Kaushambi,Uttar Pradesh,9.154069,81.509894,25.377794,06622dd4,hindi


In [41]:
day = add_res_col('md:clusterid', 'cluster', 'userid', day)

In [48]:


isin = perc.disthash.isin(day.cluster.unique())

In [ ]:
with pd.option_context('display.max_rows', None): 
    display(perc.sort_values('perc')[['perc', 'tot', 'target', 'distname']])

In [69]:
def statewide(df):
    trg = df.target.sum().astype(int)
    tot = df.tot.sum().astype(int)
    return pd.Series([trg/tot, trg, tot], index=['percentage', 'malaria', 'total'])

states = perc.groupby('state').apply(statewide).round(4)
states['total'] = states['total'].astype(int)
states['malaria'] = states['malaria'].astype(int)

states

,percentage,malaria,total
state,,,
Chhatisgarh,0.1905,64,336
Jharkhand,0.2297,136,592
Orissa,0.0698,22,315
Uttar Pradesh,0.1712,636,3716


In [109]:
perc[(perc.target >= 3) & (perc.perc > 0.2)].cluster.shape

(31,)

In [110]:
perc[(perc.target >= 3) & (perc.perc > 0.1)].cluster.shape[0] / perc.shape[0]

0.4357142857142857

In [111]:
perc[(perc.target >= 3) & (perc.perc > 0.1)].target.sum() / perc.target.sum()

0.7975609756097561

In [122]:
chosen_clusters = perc[(perc.target >= 3) & (perc.perc > 0.1)].cluster
top_cities = cities[cities.disthash.isin(chosen_clusters)].reset_index(drop=True)

In [ ]:
perc[perc.perc > .35].sort_values('target')

In [90]:
chosen_clusters = perc[(perc.perc > .35) & (perc.target >= 3)].cluster
malaria_cities = cities[cities.disthash.isin(chosen_clusters)].reset_index(drop=True)
malaria_cities = malaria_cities[~malaria_cities.state.isin(['Orissa'])]
malaria_cities.to_csv('output/malaria-cities.csv', index=False)

In [ ]:
malaria_cities

In [94]:
import pandas as pd

top_cities = pd.read_csv('output/top-cities.csv')

In [ ]:
top_cities

In [26]:

def pause_adsets(m, clusters):
    get_ad = lambda c: next((a for k, a in m.running_ads.items() if c in k), None)
    to_delete = [get_ad(c) for c in clusters]
    upd = lambda a: a.api_update(params={ 'status': 'PAUSED' })
    return [upd(ad) for ad in to_delete]
        
# res = pause_adsets(m, perc[perc.perc == 0.0].cluster)

In [ ]:
aud = get_aud(m, cnf, True)

In [ ]:
aud

In [9]:
from malaria import unsaturated

clusters = unsaturated(df, cnf)

In [75]:
adsets = m.account.get_ad_sets(fields=['lifetime_budget', 'instagram_actor_id', 'campaign_id', 'targeting'])

In [55]:
import hashlib
import hmac

def secret_proof(app_secret, access_token):
    h = hmac.new (
        app_secret.encode('utf-8'),
        msg=access_token.encode('utf-8'),
        digestmod=hashlib.sha256
    )
    return h.hexdigest()


secret = env('FACEBOOK_APP_SECRET')
token = ''
user = users[0]

proof = secret_proof(secret, token)

import requests

res = requests.get(f'https://graph.facebook.com/v7.0/{user}/ids_for_apps?access_token={token}&appsecret_proof={proof}')

res.text

'{"error":{"message":"(#100) An owning business is required for this request","type":"OAuthException","code":100,"fbtrace_id":"AypFAF9YX6tqhTqMedHqQHB"}}'

In [33]:
import time

from facebook_business.adobjects.serverside.event import Event
from facebook_business.adobjects.serverside.event_request import EventRequest
from facebook_business.adobjects.serverside.user_data import UserData
from facebook_business.adobjects.serverside.custom_data import CustomData
from facebook_business.api import FacebookAdsApi

# access_token = '<ACCESS_TOKEN>'
# pixel_id = 'ADS_PIXEL_ID>'


# user_data = UserData(
#     email='joe@eg.com',
#     fbc='fb.1.1554763741205.AbCdEfGhIjKlMnOpQrStUvWxYz1234567890',
#     fbp='fb.1.1558571054389.1098115397'
# )

# custom_data = CustomData(
#     currency='usd',
#     value=123.45
# )

# event = Event(
#     event_name='Purchase',
#     event_time=int(time.time()),
#     user_data=user_data,
#     custom_data=custom_data,
# )

# events = [event]

# event_request = EventRequest(
#     events=events,
#     pixel_id=pixel_id)

# event_response = event_request.execute()
# print(event_response)

In [6]:

cities = load_cities('output/cities.csv')



In [ ]:
from marketing import get_creatives, get_label
from facebook_business.adobjects.adcreative import AdCreative

def delete_our_creatives(account, label):
    label = get_label(account, label)
    creatives = get_creatives(account, label['id'])
    print(len(creatives))

    i = 0
    for c in creatives.values():
        try:
            c.api_delete()
            i += 1
            print(i)
        except Exception as e:
            print(e)

# delete_our_creatives(m.account, 'virtuallab-1')

In [14]:
from responses import get_forms
from datetime import datetime

survey_user = '10383123-9fb1-429b-8312-447c1b54b17a'
shortcodes = ['baselinehin', 'baselineeng']

timestamp = datetime(2020, 7, 25)

forms = list(get_forms(survey_user, shortcodes, timestamp, cnf['chatbase']))

In [15]:
questions = [
    ('malaria_incidence', 'f6e69027-97cc-494e-8d52-318b75047e23'),
    ('dist_medical', 'd7573919-8a7e-457f-9a1d-1f8c389127a7'),
    ('education', 'e40fa1c6-13a1-4a02-91cd-0eaade11864d'),
    ('owns_net', 'bd4802c6-7bdb-40f0-aac1-18cc6df7da6e'),
    ('has_ac', 'e279587c-975f-433a-adab-1ad563876af6'),
    ('has_ac', 'e279587c-975f-433a-adab-1ad563876af6'),
    ('net_lastnight', 'b5b1ff58-c8df-4890-9b1c-0cd40ce6edc0'),
    ('home', '4d0ae478-3893-4b46-ab39-d6848c69245d')]


In [16]:
from responses import get_response_df

def ref_translation(eng_form, other_form):
    vals = [(a['ref'], b['ref']) for a, b 
            in zip(eng_form['fields'], other_form['fields'])]
    lookup = dict(vals)
    rev_lookup = {v:k for k,v in lookup.items()}
    return lookup, rev_lookup


def get_filtered_responses(survey_user, eng_form, other_form, questions, db_cnf):
    ref_lookup, rev_ref_lookup = ref_translation(eng_form, other_form)
    refs = [ref_lookup[r] for _, r in questions]
    responses = get_response_df(survey_user, ['baselinehin'], refs, db_cnf)

    _, last_ref = questions[-1]
    
    answered = responses[responses.question_ref == last_ref].userid.unique()    

    return responses[responses.userid.isin(answered)].reset_index(drop=True)


rr = get_filtered_responses(survey_user, forms[0], forms[1], questions, cnf['chatbase'])

In [17]:
from clustering import add_res_cols
from forms import response_translator

def col_translators(a, b, questions):
    lookup, _ = ref_translation(a, b)

    refs = [ref for _, ref in questions]
    ts = { qt['ref']: response_translator(q, qt)
                   for qt, q in zip(a['fields'], b['fields'])
                   if qt['ref'] in refs}

    return [(n, lookup[ref], ts[ref]) for n, ref in questions]

col_names = col_translators(forms[0], forms[1], questions)
rr = add_res_cols(col_names, 'userid', rr).reset_index(drop=True)

In [18]:
rr = add_res_cols([('cluster', 'md:clusterid', lambda x: x)], 'userid', rr).reset_index(drop=True)

In [28]:
def make_row(df):
    r = df.iloc[0]
    r.timestamp = df.timestamp.max()
    return r

dd = rr.groupby('userid').apply(make_row).reset_index(drop=True)
dd = dd.merge(districts, left_on='cluster', right_on='disthash')


In [29]:
dd['under_net'] = dd.net_lastnight == 'Yes'

In [ ]:
import pandas as pd

with pd.option_context('display.max_rows', None): 
    display(dd.groupby('cluster').under_net.mean().sort_values())

In [ ]:
import seaborn as sns

sns.distplot(dd.groupby('cluster').under_net.mean())

In [558]:
# high_malaria_dists = perc[(perc.perc > .25) & (perc.target >= 3)].cluster.unique()
# low_malaria_dists = perc[(perc.perc < .07) & (perc.target >= 3)].cluster.unique()

In [ ]:
dd['dist_risk'] = 'med'
dd.loc[dd.cluster.isin(low_malaria_dists), 'dist_risk'] = 'low'
dd.loc[dd.cluster.isin(high_malaria_dists), 'dist_risk'] = 'high'

In [31]:
dd['malaria'] = dd.malaria_incidence == 'Yes'

In [32]:
def malaria_prob(groupby):
    d = groupby.malaria.value_counts(normalize=True)
    d.name = 'count'
    d = d.reset_index(level=-1)
    d = d[d.malaria == True].drop(columns='malaria')
    return d

In [33]:
def stats(df):
    targets = df.malaria.sum()
    tot = df.shape[0]
    non_target = tot - targets
    return pd.Series([targets/tot, targets, non_target, tot], index=['perc', 'target', 'non_target', 'tot'])

perc = dd.groupby('disthash').apply(stats).reset_index()

In [35]:
import pandas as pd

tops = pd.read_csv('output/top-cities.csv')

In [ ]:
dd

In [44]:
tots = dd[dd.cluster.isin(tops.disthash.unique())].groupby('cluster').count().userid.sort_values()

In [ ]:
tots

In [72]:
saturated = perc[perc.non_target > 150].disthash

56    786f66b7
Name: disthash, dtype: object

In [79]:
cities['creative_group'] = 'hindi'
cities.loc[cities.disthash.isin(saturated), 'creative_group'] = 'exclusions'

In [84]:
cities.to_csv('output/cities.csv', index=False)

In [8]:
import pandas as pd

cities = pd.read_csv('output/cities.csv')

In [ ]:
for (i,n,cg), df in cities.groupby(['disthash', 'distname', 'creative_group']):
    print(cg)

In [32]:
malaria_prob(dd.groupby('under_net')) 

,count
under_net,
False,0.192326
True,0.194123


In [522]:
malaria_prob(dd.groupby(['home']))

,count
home,
"Kutcha (made of mud, tin, straw)",0.243208
Pucca (have cement/brick wall and floor,0.168642
Semi-pucca,0.210866


In [550]:
malaria_prob(dd.groupby(['has_ac']))

,count
has_ac,
Don't know,0.141463
No,0.199307
Yes,0.156977


In [ ]:
malaria_prob(dd.groupby(['dist_risk', 'home', 'has_ac']))

In [531]:
malaria_prob(dd.groupby(['owns_net']))

,count
owns_net,
Don't know,0.066667
No,0.154300
Yes,0.207216


In [549]:
malaria_prob(dd.groupby(['dist_risk', 'owns_net']))

count
dist_risk owns_net            
high      Don't know  0.166667
          No          0.338462
          Yes         0.437055
low       No          0.137500
          Yes         0.065693
med       Don't know  0.055556
          No          0.130580
          Yes         0.191207

In [ ]:
malaria_prob(dd.groupby(['state', 'dist_risk', 'owns_net']))

In [ ]:
X = dd[['education', 'home', 'dist_medical', 'has_ac', 'owns_net']]


for col in X.columns:
    X[col] = X[col].astype('category')
    X[col] = X[col].cat.codes

In [ ]:
X

In [423]:
X.shape

(4474, 5)

In [425]:
y = dd['malaria']

In [ ]:
y.shape

In [542]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

model.fit(X[:2000], y[:2000])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:

from sklearn.tree import plot_tree

plot_tree(model)

In [300]:
dd[(dd.owns_net == 'Yes') & 
   (dd.has_ac == 'No') & 
   (dd.home == 'Kutcha (made of mud, tin, straw)')] \
   .malaria_incidence.value_counts(normalize=True)

No            0.636550
Yes           0.268994
Don’t know    0.094456
Name: malaria_incidence, dtype: float64

In [ ]:
dd

In [330]:
def times(df, mi, ma):
    return df[(df.timestamp > mi) & (df.timestamp < ma)]    


times(dd, datetime(2020,7,22,tzinfo=timezone.utc), datetime(2020,7,24,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.846106
Yes           0.113693
Don't know    0.040201
Name: has_ac, dtype: float64

In [331]:
times(dd, datetime(2020,7,24,tzinfo=timezone.utc), datetime(2020,7,25,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.797244
Yes           0.147638
Don't know    0.055118
Name: has_ac, dtype: float64

In [332]:
times(dd, datetime(2020,7,25,tzinfo=timezone.utc), datetime(2020,7,26,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.862589
Yes           0.090426
Don't know    0.046986
Name: has_ac, dtype: float64

In [328]:
times(dd, datetime(2020,7,26,tzinfo=timezone.utc), datetime(2020,7,28,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.825040
Yes           0.126806
Don't know    0.048154
Name: has_ac, dtype: float64

In [ ]:
dd

In [301]:
dd.malaria_incidence.value_counts(normalize=True)

No            0.670988
Yes           0.191775
Don’t know    0.137237
Name: malaria_incidence, dtype: float64

In [ ]:
# responses should be converted and translated...